In [1]:
from collections import Counter
from math import sqrt
from random import Random
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from nltk.stem.porter import PorterStemmer
from nltk.tokenize import word_tokenize, sent_tokenize
import string
import re
import numpy as np
from scipy.sparse import csr_matrix
from scipy.sparse import csc_matrix
from scipy.sparse import lil_matrix
from scipy.sparse.linalg import svds
#from scipy.stats.stats import spearmanr
from scipy.stats import spearmanr
from random import shuffle
import math
import random
from numpy import *
import pandas as pd
import matplotlib.pyplot as plt
import datetime
from datetime import datetime
#from sparsesvd import sparsesvd
from numpy.linalg import matrix_rank
from scipy.stats import chi2_contingency
from matplotlib.pyplot import *
from numpy import inf
from scipy.linalg import svd
from foobar import *

In [106]:
#Evaluate the verctors from GloVe of Text8 
dim = ["200", "300", "400", "500", "600"]
test_files = ['wordsim353.txt', 'men_dataset.txt', 'mturk.txt', 'rarewords.txt', 'simlex999.txt']
text8_ws_score_glove = list()
for k in range(len(test_files)):
    path_input = "D:\\carme\\test dataset\\similarities\\"+test_files[k]
    svd_score = []
    for i in dim:
        #glovevector = loadglovemodel("D:\\Correspondence analysis to PMI-based word embeddings\\GloVe-text8\\vectors"+i+".txt")
        glovevector = loadglovemodel("D:\\carme\\text8\\GloVe-master\\GloVe-master\\vectors"+i+".txt")
        #For Glove, there is extra word (<unk>) vector which is not in vacobulary. Therefore, we delete this word in word vectors: list(glovevector.keys())[-1]
        orderedNames = list(glovevector.keys())[:-1]
        glovematrix = np.array([glovevector[i] for i in orderedNames])
        #glovevocab = loadglovevocab("D:\\Correspondence analysis to PMI-based word embeddings\\GloVe-text8\\vocab.txt")
        glovevocab = loadglovevocab("D:\\carme\\text8\\GloVe-master\\GloVe-master\\vocab.txt")
        prices = range(0, glovematrix.shape[0])
        vocabsum =  list(glovevocab.values())
        vocabtuple = merge(prices, vocabsum)
        vocab = dict(zip(glovevocab.keys(), vocabtuple))

        glovematrix_vectors_words = fittingmatrixvectors(glovematrix)

        svd_score.append(ws_evaluate(glovematrix_vectors_words, vocab, mearsure = 'cosine', path = path_input))
    text8_ws_score_glove.append(svd_score)
np.save("D:\\carme\\text8\\text8_ws_score_glove.npy", text8_ws_score_glove)

In [4]:
text8_ws_score_glove

[[0.421724234956006,
  0.4133850471938503,
  0.415495605022818,
  0.41093599834870653,
  0.41102972563592133],
 [0.16767423423510677,
  0.17464811894936197,
  0.17390351884694585,
  0.17358549933867332,
  0.17285304383308447],
 [0.494036134284251,
  0.48743018474884037,
  0.4971837305368383,
  0.4979369966834748,
  0.5015415558678225],
 [0.18051858706281598,
  0.17234621852333784,
  0.18081112902025068,
  0.17752490769840132,
  0.17527442421156472],
 [0.1477221940769442,
  0.14558290458716772,
  0.14700907667348942,
  0.14790224920445078,
  0.14494012832296174]]

In [6]:
# For Glove there is extra word (<unk>) vector which is not in vacobulary. Therefore, we delete this word vector.
list(glovevector.keys())[-1]

'<unk>'

In [7]:
glovevector = loadglovemodel("D:\\carme\\text8\\GloVe-master\\GloVe-master\\vectors200.txt")
orderedNames = list(glovevector.keys())[:-1]
glovematrix = np.array([glovevector[i] for i in orderedNames])
glovevocab = loadglovevocab("D:\\carme\\text8\\GloVe-master\\GloVe-master\\vocab.txt")
prices = range(0, glovematrix.shape[0])
vocabsum =  list(glovevocab.values())
vocabtuple = merge(prices, vocabsum)
vocab = dict(zip(glovevocab.keys(), vocabtuple))
glovematrix_vectors_words = fittingmatrixvectors(glovematrix)
glove_vocab = vocab.copy()
svd_vocab = np.load("D:\\carme\\text8\\decomposed matrix\\vocab.npy", allow_pickle=True)
svd_vocab = dict(enumerate(svd_vocab.flatten(), 1))[1]

In [8]:
#To look at word pairs in Text8

test_files = ['wordsim353.txt', 'men_dataset.txt', 'mturk.txt', 'rarewords.txt', 'simlex999.txt']

text8_test_word_pairs = []

for i in range(len(test_files)):
    
    path_input = "D:\\carme\\test dataset\\similarities\\"+test_files[i]
    
    test_word_pairs_set = read_test_word_pairs_set(path_input)
    
    evaluate_word_pairs_create = evaluate_word_pairs(glovematrix_vectors_words, test_word_pairs_set, glove_vocab, "cosine")
    
    text8_test_word_pairs.append((len(test_word_pairs_set), evaluate_word_pairs_create[1], float(evaluate_word_pairs_create[1]) / len(test_word_pairs_set)))
#table 1
text8_test_word_pairs

[(353, 277, 0.7847025495750708),
 (3000, 1544, 0.5146666666666667),
 (287, 221, 0.7700348432055749),
 (2034, 205, 0.1007866273352999),
 (999, 726, 0.7267267267267268)]

In [99]:
#To detect whether the vocab in GloVe is the same as that in SVD
if len(glove_vocab)!=len(svd_vocab):
    print("Not equal")
else:
    flag=0
    for i in glove_vocab:
        if glove_vocab.get(i)[1]!=svd_vocab.get(i)[1]:
            flag=1
            break
    if flag==0:
        print("Both the dictionaries are equal")
    else:
        print("Both the dictionaries are not equal")

Both the dictionaries are equal


In [19]:
glove_vocab.get(i)

(11814, array(100.))

In [20]:
svd_vocab.get(i)

(11371, 100)

In [107]:
#Evaluate the verctors from GloVe of BNC
dim = ["200", "300", "400", "500", "600"]
test_files = ['wordsim353.txt', 'men_dataset.txt', 'mturk.txt', 'rarewords.txt', 'simlex999.txt']
bnc_ws_score_glove = list()
for k in range(len(test_files)):
    path_input = "D:\\carme\\test dataset\\similarities\\"+test_files[k]
    svd_score = []
    for i in dim:
        glovevector = loadglovemodel("D:\\carme\\bnc\\GloVe-master\\GloVe-master\\vectors"+i+".txt")
        #For Glove, there is extra word (<unk>) vector which is not in vacobulary. Therefore, we delete this word in word vectors: list(glovevector.keys())[-1]
        orderedNames = list(glovevector.keys())[:-1]
        glovematrix = np.array([glovevector[i] for i in orderedNames])
        glovevocab = loadglovevocab("D:\\carme\\bnc\\GloVe-master\\GloVe-master\\vocab.txt")
        prices = range(0, glovematrix.shape[0])
        vocabsum =  list(glovevocab.values())
        vocabtuple = merge(prices, vocabsum)
        vocab = dict(zip(glovevocab.keys(), vocabtuple))

        glovematrix_vectors_words = fittingmatrixvectors(glovematrix)

        svd_score.append(ws_evaluate(glovematrix_vectors_words, vocab, mearsure = 'cosine', path = path_input))
    bnc_ws_score_glove.append(svd_score)
np.save("D:\\carme\\bnc\\bnc_ws_score_glove.npy", bnc_ws_score_glove)

In [108]:
# For Glove there is extra word (<unk>) vector which is not in vacobulary. Therefore, we delete this word vector.
list(glovevector.keys())[-1]

'<unk>'

In [9]:
glovevector = loadglovemodel("D:\\carme\\bnc\\GloVe-master\\GloVe-master\\vectors200.txt")
orderedNames = list(glovevector.keys())[:-1]
glovematrix = np.array([glovevector[i] for i in orderedNames])
glovevocab = loadglovevocab("D:\\carme\\bnc\\GloVe-master\\GloVe-master\\vocab.txt")
prices = range(0, glovematrix.shape[0])
vocabsum =  list(glovevocab.values())
vocabtuple = merge(prices, vocabsum)
vocab = dict(zip(glovevocab.keys(), vocabtuple))
glove_vocab = vocab.copy()
svd_vocab = np.load("D:\\carme\\bnc\\decomposed matrix\\bnc_vocab.npy", allow_pickle=True)
svd_vocab = dict(enumerate(svd_vocab.flatten(), 1))[1]

In [10]:
#To look at word pairs in BNC

test_files = ['wordsim353.txt', 'men_dataset.txt', 'mturk.txt', 'rarewords.txt', 'simlex999.txt']

bnc_test_word_pairs = []

for i in range(len(test_files)):
    
    path_input = "D:\\carme\\test dataset\\similarities\\"+test_files[i]
    
    test_word_pairs_set = read_test_word_pairs_set(path_input)
    
    evaluate_word_pairs_create = evaluate_word_pairs(glovematrix_vectors_words, test_word_pairs_set, glove_vocab, "cosine")
    
    bnc_test_word_pairs.append((len(test_word_pairs_set), evaluate_word_pairs_create[1], float(evaluate_word_pairs_create[1]) / len(test_word_pairs_set)))
#table 1
bnc_test_word_pairs

[(353, 276, 0.7818696883852692),
 (3000, 1925, 0.6416666666666667),
 (287, 197, 0.686411149825784),
 (2034, 204, 0.10029498525073746),
 (999, 847, 0.8478478478478478)]

In [11]:
if len(glove_vocab)!=len(svd_vocab):
    print("Not equal")
else:
    flag=0
    for i in glove_vocab:
        if i.isascii():
            if glove_vocab.get(i)[1]!=svd_vocab.get(i)[1]:
                flag=1
                break
    if flag==0:
        print("Both the dictionaries are equal")
    else:
        print("Both the dictionaries are not equal")

Both the dictionaries are equal


In [12]:
non_ascii_glove = [ ]
for i in glove_vocab:
    if not i.isascii():
        non_ascii_glove.append(i)
print(len(non_ascii_glove))

non_ascii_glove_list = list()

for i in non_ascii_glove:
    non_ascii_glove_list.append(glove_vocab.get(i)[1])

32


In [13]:
non_ascii_svd = [ ]
for i in svd_vocab:
    if not i.isascii():
        non_ascii_svd.append(i)
print(len(non_ascii_svd))

non_ascii_svd_list = list()

for i in non_ascii_svd:
    non_ascii_svd_list.append(svd_vocab.get(i)[1])
non_ascii_svd_list.sort(reverse=True)

32


In [14]:
j = 0
for i in range(len(non_ascii_svd)):
    if non_ascii_glove_list[i] == non_ascii_svd_list[i]:
        j = j+1
j

32